In [5]:
import functools
import re

import numpy as np
import json
import pandas as pd
from fuzzywuzzy import process

import auxiliar_3

/Users/erickavendanogarcia/Downloads/servicio_social/venv/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
!pip install fuzzywuzzy

  Obtaining dependency information for fuzzywuzzy from https://files.pythonhosted.org/packages/43/ff/74f23998ad2f93b945c0309f825be92e04e0348e062026998b5eefef4c33/fuzzywuzzy-0.18.0-py2.py3-none-any.whl.metadata

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


# Encuesta

In [11]:
encuesta = pd.read_csv('/Users/erickavendanogarcia/Downloads/servicio_social/encuestas/Zoom3/datos/hermanos.csv')
encuesta.columns = encuesta.iloc[1]
encuesta = encuesta.drop(encuesta.index[0:1])
encuesta = encuesta.reset_index(drop=True)

/var/folders/hb/c3zh8bns0mx4nw98_6k7rx8m0000gn/T/ipykernel_81688/1377798677.py:1: DtypeWarning: Columns (0,3,5,9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  encuesta = pd.read_csv('/Users/erickavendanogarcia/Downloads/servicio_social/encuestas/Zoom3/datos/hermanos.csv')


In [12]:
encuesta.head(3)

1,id,token,submitdate,lastpage,startlanguage,seed,startdate,datestamp,fecha,folio,...,Hermano5TrabajoActEl,Hermano5TrabajoOfici,hijos,cuantoshijos,cuantoshijos_other,sobrehijo1,q0r921,q0r94,q0r308,q0r79
0,id,token,submitdate,lastpage,startlanguage,seed,startdate,datestamp,fecha,folio,...,Hermano5TrabajoActEl,Hermano5TrabajoOfici,hijos,cuantoshijos,cuantoshijos_other,sobrehijo1,q0r921,q0r94,q0r308,q0r79
1,34,{question_not_shown},29/03/2021 12:08,1,es-MX,1234036385,29/03/2021 11:49,29/03/2021 12:08,29/03/2021 00:00,5240,...,{question_not_shown},{question_not_shown},N,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
2,35,{question_not_shown},29/03/2021 12:46,1,es-MX,1582248881,29/03/2021 12:37,29/03/2021 12:46,29/03/2021 00:00,5220,...,{question_not_shown},{question_not_shown},N,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}


# Diccionario de datos

Este diccionario contiene toda la información al respecto de las preguntas hechas en la encuesta de LimeSurvey, además de las descripciones a las preguntas, entre otras cosas

In [15]:
diccionario = pd.read_excel('/Users/erickavendanogarcia/Downloads/servicio_social/encuestas/Zoom3/datos/Diccionario Zoom 3 hermanos 11.04.2022 DW.xlsx')
diccionario.head(3)

,item order,related,alive,new_code,code,original item lime survey,question,options,Unnamed: 8,Unnamed: 9,...,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29
0,1.0,G,_,NaN,id,ID de la respuesta,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,G,_,NaN,submitdate,Fecha en que se envió,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,G,_,NaN,lastpage,Ultima pagina,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
diccionario.columns

Index(['item order', 'related', 'alive', 'new_code', 'code',
       'original item lime survey', 'question', 'options', 'Unnamed: 8',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
       'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24',
       'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28',
       'Unnamed: 29'],
      dtype='object')

In [17]:
diccionario_2 = diccionario.set_index('code', inplace=True)


In [18]:
import auxiliar_4

# Funciones auxiliares

In [ ]:
mapear_codigos = functools.partial(auxiliar_4.mapear_codigos, encuesta, mapeo_respuestas)
llenar_respuesta_libre = functools.partial(auxiliar_4.llenar_respuesta_libre, encuesta)
llenar_vacios = functools.partial(auxiliar_4.llenar_vacios, encuesta)
mostrar_unicos = functools.partial(auxiliar_4.mostrar_unicos, encuesta)
contar_faltantes = functools.partial(auxiliar_4.contar_faltantes, encuesta)
unificar_observaciones = functools.partial(auxiliar_4.unificar_observaciones, encuesta)

# Mapeo de enfermedades

Primero, de la encuesta, vamos a obtener todas las columnas que estén dentro de ese dataset, para luego poder filtrar y quedarnos con todas aquellas que contengan [other], y no pertenezcan a las ya mapeadas en el .json de edades

In [10]:
columnas = encuesta.columns.tolist()
columnas

['id',
 'token',
 'submitdate',
 'lastpage',
 'startlanguage',
 'seed',
 'startdate',
 'datestamp',
 'fecha',
 'folio',
 'encuestador',
 'encuestador_other',
 'Cuantoshermanos',
 'gs566',
 'hermano1',
 'q0r949',
 'q0r576',
 'q0r576_other',
 'q0r445_SQ002',
 'q0r445_SQ003',
 'q0r445_SQ004',
 'q0r445_SQ005',
 'q0r445_other',
 'q0r326',
 'q0r326_other',
 'q0r440_SQ002',
 'q0r440_SQ003',
 'q0r440_SQ004',
 'q0r440_SQ005',
 'q0r440_other',
 'q0r972',
 'q0r972_other',
 'q0r969_SQ002',
 'q0r969_SQ003',
 'q0r969_SQ004',
 'q0r969_SQ005',
 'q0r969_other',
 'q0r512',
 'q0r512_other',
 'q0r357_SQ002',
 'q0r357_SQ003',
 'q0r357_SQ004',
 'q0r357_SQ005',
 'q0r357_other',
 'q0r984',
 'q0r984_other',
 'q0r532_SQ002',
 'q0r532_SQ003',
 'q0r532_SQ004',
 'q0r532_SQ005',
 'q0r532_other',
 'q0r348',
 'q0r348_other',
 'q0r843_SQ002',
 'q0r843_SQ003',
 'q0r843_SQ004',
 'q0r843_SQ005',
 'q0r843_other',
 'q0r272',
 'q0r272_other',
 'q0r1012_SQ002',
 'q0r1012_SQ003',
 'q0r1012_SQ005',
 'q0r1012_SQ006',
 'q0r1012_

In [11]:
columnas_filtradas = [col for col in columnas if '_other' in col]
columnas_filtradas

['encuestador_other',
 'q0r576_other',
 'q0r445_other',
 'q0r326_other',
 'q0r440_other',
 'q0r972_other',
 'q0r969_other',
 'q0r512_other',
 'q0r357_other',
 'q0r984_other',
 'q0r532_other',
 'q0r348_other',
 'q0r843_other',
 'q0r272_other',
 'q0r1012_other',
 'q0r253_other',
 'q0r253_othercomment',
 'q0r160_other',
 'q0r552_other',
 'q0r1005_other',
 'q0r291_other',
 'q0r337_other',
 'q0r263_other',
 'q0r50_other',
 'q0r194_other',
 'q0r774_other',
 'q0r538_other',
 'q0r796_other',
 'q0r666_other',
 'q0r536_other',
 'q0r652_other',
 'q0r809_other',
 'q0r809_othercomment',
 'q0r981_other',
 'q0r353_other',
 'DEPORTE2222_other',
 'Artista1_other',
 'CHOFER56_other',
 'fORMA31_other',
 'INFO_other',
 'EMPRESS_other',
 'VIRTUAA_other',
 'CLIENTT_other',
 'ADMINISS_other',
 'GUBB_other',
 'EDUKTV_other',
 'INEP_other',
 'SEG_other',
 'SALUD21_other',
 'CONSTRUCC_other',
 'fabhj22_other',
 'AGRUCUT_other',
 'q0r726_other',
 'q0r875_other',
 'q0r81_other',
 'q0r1013_other',
 'q0r792_other',

## Estudios

In [12]:
columnas_estudio = diccionario[diccionario['new_code'].str.contains('estudio', case=False, na=False)]
# Crear un diccionario que mapee las columnas "code" y "new_code" para estas columnas específicas relacionadas con el trabajo
diccionario_estudio = dict(zip(columnas_estudio['code'], columnas_estudio['new_code']))
# Mostrar algunos elementos del diccionario para verificar el mapeo
list(diccionario_estudio.items())[:10]

[('q0r272', 'estudios'),
 ('q0r272[other]', 'estudios_alternativo'),
 ('q0r1012[SQ002]', 'estudio_b'),
 ('q0r1012[SQ003]', 'estudio_h'),
 ('q0r1012[SQ005]', 'estudio_e'),
 ('q0r1012[SQ006]', 'estudio_m'),
 ('q0r1012[SQ007]', 'estudio_p'),
 ('q0r1012[SQ008]', 'estudio_g'),
 ('q0r1012[SQ009]', 'estudio_n'),
 ('q0r1012[other]', 'estudio_o')]

In [13]:
encuesta_estudios = ['q0r652_other']

In [14]:
columnas_filtradas_2 = [col for col in columnas_filtradas if col in encuesta_estudios]
columnas_filtradas_2

['q0r652_other']

In [15]:
# Obtener los valores de las columnas filtradas
valores_columnas = encuesta[columnas_filtradas_2].values.flatten().tolist()

# Eliminar valores nulos o vacíos
valores_columnas = [valor for valor in valores_columnas if pd.notnull(valor) and valor != '']

# Convertir los valores a minúsculas
valores_columnas = [valor.lower() for valor in valores_columnas]

# Filtrar para eliminar '{question_not_shown}'
valores_columnas = [valor for valor in valores_columnas if valor != '{question_not_shown}' and valor not in columnas_filtradas_2]

# Obtener valores únicos
estudios_unicos = list(set(valores_columnas))

print("Lista original de estudios:")
print(estudios_unicos)

Lista original de estudios:
['ego', 'ultrasonidos', 'trigliceridos', 'encefalogramas', 'hormonales', 'prostata', 'radiografia', 'ultrasonido pelvico', 'prueba covid', 'dentista', 'dental y dermatologico', 'glucosa', 'encefalograma', 'electroencefalograma', 'rayos x', 'radiografias', 'tiroides']


In [16]:
estudios_generales = [
    "Orina (análisis de orina)",
    "Tomografía computarizada (TC)",
    "Electroencefalograma (EEG)",
    "Radiografía de tórax",
    "Estudios de tiroides",
    "Prueba de antígeno prostático específico (PSA)",
    "Ultrasonido renal (USG renal)",
    "Cultivo de heces",
    "Biometría hemática completa",
    "Colesterol total, LDL, HDL",
    "Prueba de tolerancia a la glucosa",
    "Mamografía",
    "Densitometría ósea",
    "Colonoscopia",
    "Prueba de Papanicolaou",
    "Electrocardiograma (ECG)",
    "Ultrasonido abdominal",
    "Ultrasonido pélvico",
    "Vitamina D",
    "Ácido úrico",
    "Perfil hepático",
    "Perfil renal",
    "Perfil lipídico",
    "Resonancia magnética (RM)"
]


In [17]:
def encontrar_relacion_fuzzy(estudio):
    match, score = process.extractOne(estudio, estudios_generales)
    # Puedes ajustar el umbral de puntuación según tus necesidades
    if score > 80:
        return match
    return ''

df_relacion_fuzzy = pd.DataFrame({
    'clave': estudios_unicos,
    'valor': [encontrar_relacion_fuzzy(estudio) for estudio in estudios_unicos]
})

# Mostrar el resultado
df_relacion_fuzzy


,clave,valor
0,ego,
1,ultrasonidos,Ultrasonido renal (USG renal)
2,trigliceridos,
3,encefalogramas,Electroencefalograma (EEG)
4,hormonales,
5,prostata,
6,radiografia,Radiografía de tórax
7,ultrasonido pelvico,Ultrasonido pélvico
8,prueba covid,Prueba de antígeno prostático específico (PSA)
9,dentista,


In [18]:
df_relacion_fuzzy.to_excel(r"C:\Users\Elvira Magallanes\Documents\Servicio Social\encuestas\Zoom3\output\lista_estudios_Z3.xlsx", index=False)

## Enfermedades

In [19]:
encuesta_enfermedades = ['q0r809_other', 'q0r1012_other']
columnas_filtradas_2 = [col for col in columnas_filtradas if col in encuesta_enfermedades]
columnas_filtradas_2

['q0r1012_other', 'q0r809_other']

In [20]:
# Obtener los valores de las columnas filtradas
valores_columnas = encuesta[columnas_filtradas_2].values.flatten().tolist()

# Eliminar valores nulos o vacíos
valores_columnas = [valor for valor in valores_columnas if pd.notnull(valor) and valor != '']

# Convertir los valores a minúsculas
valores_columnas = [valor.lower() for valor in valores_columnas]

# Filtrar para eliminar '{question_not_shown}'
valores_columnas = [valor for valor in valores_columnas if valor != '{question_not_shown}' and valor not in columnas_filtradas_2]

# Obtener valores únicos
enfermedades_unicas = list(set(valores_columnas))

print("Lista original de enfermedades:")
print(enfermedades_unicas)

Lista original de enfermedades:
['sindrome de gilbert', 'apendicitis', 'ovario polilquistico', 'disautonomia', 'miopia', 'trastorno de personalidad', 'dibaetes gestacional', 'hipotiroidismo', 'tiroides', 'colitis', 'gastritis', 'tdah', 'tda', 'enfermedad autoinmune kikuchi-fujimoto', 'transtorno del espectro autista', 'ovario poliquistico', 'quistes en cenos', 'tea', 'asma', 'anemia', 'ninguno', 'sindrome de lennox-gastaut', 'opq', 'alergias', 'escoliosis', 'vagotonismo', 'ninguna', 'déficit de atención', 'hipertiroidismo', 'hernia hiatal', 'desnutricion', 'roseola', 'ataques de panico', 'epilepsia', 'hipotension', 'qt largo', 'vitiligo', 'ansiedad social', 'sincope vasovagal', 'alergias ambientales']


In [21]:
enfermedades_estandar = [
    "Sobrepeso u obesidad (Dob)","Diabetes (Ddi)","Prediabetes (Dpd)","Síndrome metabólico (Dsm)","Hipertensión (Dh)","Colesterol alto (Dc)",
    "Triglicéridos altos (Dtr)","Cáncer (Dcan)","Algún problema cardiovascular (Dcar)","Algún problema respiratorio (Dres)","COVID19 (Dcov)",
    "Alcoholismo (Dal)","Tabaquismo (Dta)","Trastorno de la alimentación (bulimia, anorexia) (Dba)","Trastorno de atracón (Datr)",
    "Ansiedad (Dan)","Depresión (Dde)","Trastorno obsesivo compulsivo (Dtoc)",'Síndrome del ovario poliquístico (SOP)',
    'Alergias','Alzheimer','Apendicitis','Artritis','Cataratas','Anemia','Cirrosis hepática','Epilepsia','Esclerosis múltiple',
    'Esquizofrenia','Gastritis','Glaucoma','Hepatitis','Herpes','Hipertiroidismo','Hipotiroidismo','Insuficiencia renal']

In [22]:
def encontrar_relacion_fuzzy(enfermedad):
    match, score = process.extractOne(enfermedad, enfermedades_estandar)
    # Puedes ajustar el umbral de puntuación según tus necesidades
    if score > 80:
        return match
    return ''

In [23]:
df_relacion_enfermedades = pd.DataFrame({
    'clave': enfermedades_unicas,
    'valor': [encontrar_relacion_fuzzy(enfermedad) for enfermedad in enfermedades_unicas]
})

# Mostrar el resultado
df_relacion_enfermedades

,clave,valor
0,sindrome de gilbert,"Trastorno de la alimentación (bulimia, anorexi..."
1,apendicitis,Apendicitis
2,ovario polilquistico,Síndrome del ovario poliquístico (SOP)
3,disautonomia,
4,miopia,
5,trastorno de personalidad,"Trastorno de la alimentación (bulimia, anorexi..."
6,dibaetes gestacional,
7,hipotiroidismo,Hipotiroidismo
8,tiroides,
9,colitis,


In [24]:
df_relacion_enfermedades.to_excel(r"C:\Users\Elvira Magallanes\Documents\Servicio Social\encuestas\Zoom3\output\lista_enfermedades_Z3.xlsx", index=False)

# Mapeo de edades

In [ ]:
# Cargando el archivo JSON
with open('C:\\Users\\Elvira Magallanes\\Documents\\Servicio Social\\encuestas\\Zoom2\\output\\edades.json', 'r') as file:
    age_ranges_dict = json.load(file)

In [ ]:
# Identificar las columnas [other] que se relacionan con las edades en la encuesta
age_related_other_columns = [col for col in encuesta.columns if '[other]' in col and col.split('[')[0] in age_ranges_dict]

In [ ]:
# Verificando los primeros elementos de age_ranges_dict y las primeras columnas de survey_df
age_ranges_dict[:5], list(encuesta.columns)[:20]

In [ ]:
# Ajustando age_ranges_dict para ser un diccionario plano en lugar de una lista de diccionarios
flat_age_ranges_dict = {}
for entry in age_ranges_dict:
    flat_age_ranges_dict.update(entry)

# Verificando la estructura del diccionario ajustado
dict(list(flat_age_ranges_dict.items())[:5])

In [ ]:
# Función para verificar si un valor es numérico
def is_numeric(val):
    if isinstance(val, (int, float)):
        return True
    if isinstance(val, str) and val.replace('.', '', 1).isdigit():
        return True
    return False

In [ ]:
# Eliminar columnas [other] que no tienen datos y corregir las respuestas no numéricas
columns_not_found = []  # Lista para guardar columnas que no se encuentran
for col in age_related_other_columns:
    try:
        # Si todos los valores en la columna son NaN, eliminar la columna
        if encuesta[col].isna().all():
            encuesta.drop(columns=[col], inplace=True)
        else:
            # Reemplazar valores no numéricos con 99
            encuesta[col] = encuesta[col].apply(lambda x: x if is_numeric(x) else 99)
    except KeyError:
        columns_not_found.append(col)

columns_not_found

In [ ]:
def extract_min_max_from_string(s):
    """
    Extracts minimum and maximum values from a string using regex.
    """
    numbers = re.findall(r'(\d+\.*\d*)', s)
    if len(numbers) == 1:
        return float(numbers[0]), float(numbers[0])
    elif len(numbers) > 1:
        return float(numbers[0]), float(numbers[1])
    return None, None

In [ ]:
def regex_map_age_to_correct_column(row):
    for question_key, age_ranges in flat_age_ranges_dict.items():
        other_column = f"{question_key}[other]"
        
        # If the [other] column is not present or its value is NaN, continue with the next iteration
        if other_column not in row or pd.isna(row[other_column]):
            continue

        age_value_str = str(row[other_column])
        age_min, age_max = extract_min_max_from_string(age_value_str)

        # If no valid age values are found, continue with the next iteration
        if age_min is None or age_max is None:
            continue

        # Map the age range to the correct columns
        for age_range in age_ranges:
            defined_age_min, defined_age_max = age_range['min'], age_range['max']
            var = age_range['var']
            
            if (defined_age_min <= age_min <= defined_age_max) or (defined_age_min <= age_max <= defined_age_max):
                correct_column = f"{question_key}[{var}]"
                row[correct_column] = "Y"

    return row

In [ ]:
# Apply the regex-based function to the DataFrame
regex_corrected_survey_df = encuesta.apply(regex_map_age_to_correct_column, axis=1)

In [ ]:
# Mostrando las primeras filas del DataFrame corregido
regex_corrected_survey_df.head()

In [ ]:
#regex_corrected_survey_df.to_csv(r"C:\Users\Elvira Magallanes\Documents\Servicio Social\encuestas\Zoom2\output\encuesta_limpia.csv", index=False)

# Mapeo de trabajos

Filtraremos todas las columnas bases 

In [ ]:
columnas_trabajo = traductor[traductor['new_code'].str.contains('trabajo', case=False, na=False)]
# Crear un diccionario que mapee las columnas "code" y "new_code" para estas columnas específicas relacionadas con el trabajo
diccionario_trabajo = dict(zip(columnas_trabajo['code'], columnas_trabajo['new_code']))
# Mostrar algunos elementos del diccionario para verificar el mapeo
list(diccionario_trabajo.items())[:10]

In [ ]:
# Modificar las claves del diccionario de trabajo para reflejar la diferencia en la nomenclatura
diccionario_trabajo_modificado = {key.replace('_other', '[other]'): value for key, value in diccionario_trabajo.items()}

# Seleccionar solo las columnas que existen en la encuesta y en el diccionario de trabajo modificado
columnas_existentes_modificadas = [col for col in diccionario_trabajo_modificado.keys() if col in encuesta.columns]
columnas_existentes_modificadas

In [ ]:
# Crear un DataFrame para almacenar las respuestas relacionadas con el trabajo
respuestas_trabajo_df = encuesta[columnas_existentes_modificadas]

In [ ]:
# Renombrar las columnas usando el diccionario de trabajo modificado
respuestas_trabajo_df.columns = respuestas_trabajo_df.columns.map(lambda col: diccionario_trabajo_modificado.get(col, col))

# Limpiar las respuestas, eliminando los espacios en blanco y convirtiendo a minúsculas
respuestas_trabajo_df = respuestas_trabajo_df.apply(lambda x: x.str.strip().str.lower())

# Llenar los valores vacíos con 'ninguna'
respuestas_trabajo_df = respuestas_trabajo_df.fillna('ninguna')

In [ ]:
respuestas_trabajo_df

In [ ]:
for idx, columna in enumerate(respuestas_trabajo_df.columns):
    valores_unicos = respuestas_trabajo_df.iloc[:, idx].astype(str).unique()
    print(f"Los valores únicos en la columna '{columna}' (índice {idx}) son: {valores_unicos}")

Agregamos el JSON códigos de ocupaciones

In [ ]:
with open('C:\\Users\\Elvira Magallanes\\Documents\\Servicio Social\\encuestas\\codigos_preguntas\\codigos_ocupaciones.json', 'r', encoding='utf-8') as file:
    codigos_ocupaciones = json.load(file)

In [ ]:
from difflib import get_close_matches

# Filtrar las columnas [other] relacionadas con trabajos para cada familiar
columnas_other_familiares = [col for col in encuesta.columns if any(relative in col for relative in relatives) and "_other" in col]

# Mapear ocupaciones escritas incorrectamente a las ocupaciones correctas
for col in columnas_other_familiares:
    for idx, value in encuesta[col].dropna().iteritems():
        # Buscar coincidencias cercanas en el archivo JSON de códigos de ocupaciones
        matches = get_close_matches(value, codigos_ocupaciones.values(), n=1, cutoff=0.6)
        if matches:
            encuesta.at[idx, 'trabajo'] = list(codigos_ocupaciones.keys())[list(codigos_ocupaciones.values()).index(matches[0])]

# Mostrar las primeras filas de la columna 'trabajo' después del mapeo
encuesta['trabajo'].head()

### Limpieza Hermano 1

In [23]:
from auxiliar_3 import *

In [25]:
encuesta

1,id,token,submitdate,lastpage,startlanguage,seed,startdate,datestamp,fecha,folio,...,Hermano5TrabajoActEl,Hermano5TrabajoOfici,hijos,cuantoshijos,cuantoshijos_other,sobrehijo1,q0r921,q0r94,q0r308,q0r79
0,id,token,submitdate,lastpage,startlanguage,seed,startdate,datestamp,fecha,folio,...,Hermano5TrabajoActEl,Hermano5TrabajoOfici,hijos,cuantoshijos,cuantoshijos_other,sobrehijo1,q0r921,q0r94,q0r308,q0r79
1,34,{question_not_shown},29/03/2021 12:08,1,es-MX,1234036385,29/03/2021 11:49,29/03/2021 12:08,29/03/2021 00:00,5240,...,{question_not_shown},{question_not_shown},N,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
2,35,{question_not_shown},29/03/2021 12:46,1,es-MX,1582248881,29/03/2021 12:37,29/03/2021 12:46,29/03/2021 00:00,5220,...,{question_not_shown},{question_not_shown},N,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
3,36,{question_not_shown},29/03/2021 14:39,1,es-MX,1015102678,29/03/2021 14:33,29/03/2021 14:39,29/03/2021 00:00,5108,...,{question_not_shown},{question_not_shown},N,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
4,37,{question_not_shown},29/03/2021 18:29,1,es-MX,951047001,29/03/2021 18:29,29/03/2021 18:29,29/03/2021 00:00,5299,...,{question_not_shown},{question_not_shown},N,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713,768,{question_not_shown},29/10/2022 17:32,1,es-MX,468036015,29/10/2022 17:29,29/10/2022 17:32,29/10/2022 00:00,9027,...,{question_not_shown},{question_not_shown},N,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
714,769,{question_not_shown},29/10/2022 18:23,1,es-MX,816788322,29/10/2022 18:20,29/10/2022 18:23,29/10/2022 00:00,9028,...,{question_not_shown},{question_not_shown},N,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
715,770,{question_not_shown},29/10/2022 20:26,1,es-MX,1365617642,29/10/2022 20:23,29/10/2022 20:26,30/10/2022 00:00,9029,...,{question_not_shown},{question_not_shown},N,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
716,771,{question_not_shown},31/10/2022 19:31,1,es-MX,613937550,31/10/2022 19:28,31/10/2022 19:31,31/10/2022 00:00,9030,...,{question_not_shown},{question_not_shown},N,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}


In [26]:
mostrar_unicos(encuesta,"q0r353")

0                    -oth-
1                       10
2                       11
3                       12
4                       15
5                       16
6                       17
7                       18
8                       19
9                        2
10                      20
11                      21
12                      22
13                      23
14                      24
15                       3
16                       4
17                       5
18                       6
19                       7
20                       8
21                       9
22                  q0r353
23    {question_not_shown}
24                     NaN
Name: q0r353, dtype: object